In [1]:
import pandas as pd

finetuned_dirname = "40-epoch-distilbert-base-finetuned-phemernr2-tf"

data = pd.read_csv("../../data/processed/phemernr2-tf_dataset.csv", sep=",")
data = data[['tweet_text', 'tvt2', 'label']]
data['tweet_text'] = data['tweet_text'].str.lower()
print(data.shape)
data.head()

(1705, 3)


,tweet_text,tvt2,label
0,breaking - a germanwings airbus a320 plane rep...,training,True
1,reports that two of the dead in the #charliehe...,training,True
2,'no survivors' in #germanwings crash says fren...,training,False
3,tragedy mounts as soldier shot this am dies of...,training,True
4,watch the moment gunfire and explosions were h...,training,True


In [2]:
combined_data = data

In [3]:
import torch

class CustomTextDataset(torch.utils.data.dataset.Dataset):

    def __init__(self, texts, labels):
        self.labels = labels
        self.texts = texts
        self.attention_mask = None
        self.input_ids = None
        self.token_type_ids = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "text": self.texts[idx],
            "label": self.labels[idx],
            "attention_mask": self.attention_mask[idx] if self.attention_mask else None,
            "input_ids": self.input_ids[idx] if self.input_ids else None,
#             "token_type_ids": self.token_type_ids[idx] if self.token_type_ids else None
        }
        return sample
    
    def tokenize(self, tokenizer):
        self.attention_mask = []
        self.input_ids = []
        self.token_type_ids = []

        for text in self.texts:
            token = tokenizer(text, padding="max_length", truncation=True)
            
            self.attention_mask.append(token['attention_mask'])
            self.input_ids.append(token['input_ids'])
#             self.token_type_ids.append(token['token_type_ids'])

c:\Users\msi_b\anaconda3\envs\pytorch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
labels = []

labels_str = combined_data['label'].unique().tolist()
for i, d in combined_data.iterrows():
    lab = labels_str.index(d['label'])
    labels.append(lab)
    
print(len(labels))
labels[:10]

1705


[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]

In [5]:
train_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'])
test_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'])
train_dataset[0]

{'text': 'breaking - a germanwings airbus a320 plane reportedly crashed in the region of digne (french alps) #flightradar24 - french tv #itele',
 'label': 0,
 'attention_mask': None,
 'input_ids': None}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [7]:
# inputs = tokenizer(["you're stuck in a timewrap from 2004 though", "summa lumma dumma lumma"], padding="max_length", truncation=True)
# for k,v in inputs.items():
#     print(k)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset.tokenize(tokenizer)
test_dataset.tokenize(tokenizer)

In [9]:
print(len(train_dataset))
print(len(test_dataset))

1176
371


### Fine Tuning

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased",
                                                           output_hidden_states=False,
                                                           num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

epochs = 40
batch_size = 8
save_steps = (round((len(train_dataset)/batch_size) + 0.49)) * epochs
# save_steps = 1_000_000

training_args = TrainingArguments(
    output_dir=f"../../data/models/{finetuned_dirname}",
    num_train_epochs=epochs,
    save_steps=save_steps,
    logging_steps=300,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

print(f"Save Steps : {save_steps}")

Save Steps : 5880


In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\msi_b\AppData\Local\Temp\ipykernel_17896\3043712826.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
import time

start = time.time()

trainer.train()

print(f"Execution Time : {round(time.time() - start)} seconds")

  2%|▎         | 147/5880 [00:25<15:19,  6.24it/s]

{'loss': 0.5557, 'learning_rate': 9.75e-06, 'epoch': 1.0}


                                                  
  3%|▎         | 148/5880 [00:27<1:22:23,  1.16it/s]

{'eval_loss': 0.42117857933044434, 'eval_accuracy': 0.8274932614555256, 'eval_runtime': 2.3316, 'eval_samples_per_second': 159.12, 'eval_steps_per_second': 20.158, 'epoch': 1.0}


  5%|▌         | 294/5880 [00:51<15:03,  6.19it/s]  

...

{'eval_loss': 0.9428479671478271, 'eval_accuracy': 0.9002695417789758, 'eval_runtime': 2.3021, 'eval_samples_per_second': 161.157, 'eval_steps_per_second': 20.416, 'epoch': 39.0}


100%|██████████| 5880/5880 [17:15<00:00,  6.26it/s]

{'loss': 0.0, 'learning_rate': 0.0, 'epoch': 40.0}


                                                   
100%|██████████| 5880/5880 [17:18<00:00,  5.66it/s]

{'eval_loss': 0.94346022605896, 'eval_accuracy': 0.9002695417789758, 'eval_runtime': 2.2977, 'eval_samples_per_second': 161.467, 'eval_steps_per_second': 20.455, 'epoch': 40.0}
{'train_runtime': 1038.244, 'train_samples_per_second': 45.307, 'train_steps_per_second': 5.663, 'train_loss': 0.039785250794533704, 'epoch': 40.0}
Execution Time : 1038 seconds


In [15]:
trainer.evaluate()

100%|██████████| 47/47 [00:02<00:00, 20.98it/s]


{'eval_loss': 0.94346022605896,
 'eval_accuracy': 0.9002695417789758,
 'eval_runtime': 2.2891,
 'eval_samples_per_second': 162.073,
 'eval_steps_per_second': 20.532,
 'epoch': 40.0}